In [ ]:
import zipfile
import os

zip_path = "saved_experiments/YOUR_EXPERIMENT_NAME.zip"  # <-- change this
extract_dir = zip_path.replace(".zip", "")

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("Extracted to:", extract_dir)

In [ ]:
import json
import joblib
import pandas as pd
import numpy as np
import torch

# Load config
with open(os.path.join(extract_dir, "experiment_config.json")) as f:
    config = json.load(f)

architecture = config["architecture"]
training = config["training"]

print("Architecture:", architecture)

# Load scaler
scaler = joblib.load(os.path.join(extract_dir, "scaler.pkl"))

# Load dataset snapshot
df = pd.read_csv(os.path.join(extract_dir, "dataset_snapshot.csv"))

# Load splits
train_reacts = np.load(os.path.join(extract_dir, "train_reactions.npy"))
val_reacts   = np.load(os.path.join(extract_dir, "val_reactions.npy"))
test_reacts  = np.load(os.path.join(extract_dir, "test_reactions.npy"))

print("Dataset loaded:", df.shape)

In [ ]:
import pennylane as qml
import torch.nn as nn

n_qubits = architecture["n_qubits"]
n_layers = architecture["n_layers"]

dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev, interface="torch", diff_method="backprop")
def qnode_single(weights, x):
    qml.templates.AngleEmbedding(x, wires=range(n_qubits))
    qml.templates.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

class QuantumRegressor(nn.Module):
    def __init__(self, in_dim):
        super().__init__()
        self.encoder = nn.Linear(in_dim, n_qubits)
        self.q_weights = nn.Parameter(torch.randn(n_layers, n_qubits, 3))
        self.fc1 = nn.Linear(n_qubits, architecture["fc1_output_dim"])
        self.fc2 = nn.Linear(architecture["fc1_output_dim"], 1)

    def forward(self, x):
        x = torch.tanh(self.encoder(x))
        q_out = torch.stack(qnode_single(self.q_weights, x), dim=1)
        h = torch.relu(self.fc1(q_out))
        return self.fc2(h)

model = QuantumRegressor(in_dim=architecture["encoder_input_dim"])
model.load_state_dict(torch.load(os.path.join(extract_dir, "best_model.pt"), map_location="cpu"))
model.eval()

print("Model restored.")

In [ ]:
features_train = [...]  # paste your 29 feature names here

X = df[features_train].values.astype(np.float32)
y = df["delta_log10_S"].values.astype(np.float32).reshape(-1,1)

X_scaled = scaler.transform(X)

train_mask = df["Reaction"].isin(train_reacts)
val_mask   = df["Reaction"].isin(val_reacts)
test_mask  = df["Reaction"].isin(test_reacts)

X_train = X_scaled[train_mask]
y_train = y[train_mask]

X_val = X_scaled[val_mask]
y_val = y[val_mask]

X_test = X_scaled[test_mask]
y_test = y[test_mask]

print("Splits reconstructed.")

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error

def evaluate(X, y):
    with torch.no_grad():
        preds = model(torch.tensor(X, dtype=torch.float32)).numpy()
    mse = mean_squared_error(y, preds)
    r2 = r2_score(y, preds)
    return mse, r2

print("TRAIN:", evaluate(X_train, y_train))
print("VAL:", evaluate(X_val, y_val))
print("TEST:", evaluate(X_test, y_test))

In [ ]:
def extract_embeddings(X):
    embeddings = []
    with torch.no_grad():
        for i in range(len(X)):
            x = torch.tensor(X[i:i+1], dtype=torch.float32)
            x_enc = torch.tanh(model.encoder(x))
            q_out = torch.stack(qnode_single(model.q_weights, x_enc), dim=1)
            embeddings.append(q_out.numpy())
    return np.vstack(embeddings)

quantum_embeddings = extract_embeddings(X_scaled)

print("Embedding shape:", quantum_embeddings.shape)

In [ ]:
#then same analysis